# Sequential removal of links and resiliency testing

- Disruption of a network by removal of links, based on:
    + Sum of betweenness centrality of from and to nodes
    + Volume of commodity flow
- Calculation of performance in terms of cost and unmet demand by re-running disrupted network. 
- Plot link removal along x-axis and performance on y-axis, comparing networks of differing evenness. Dynamic report generated in an RMarkdown automatically from this Notebook.

**Assumptions**

- Working in a Python 3.x environment for this notebook
    + Refer to the README in this repository for instructions on setup of all dependencies with `conda`
- Access to ArcGIS license server if necessary
- FTOT scenario was run with Network Density Reduction (NDR) off
    + NDR_On should be False in the scenario XML file

*Reference*

- [NetworkX Documentation](https://networkx.github.io/documentation/stable/tutorial.html)

In [1]:
import pandas as pd
import geopandas as gpd
import sqlalchemy
import networkx as nx
import os
import pickle
import momepy # for conversion from geopandas GeoDataFrame to networkX Graph
import subprocess
import shutil
import webbrowser
import resiliency_disruptions

# Uses Reference Scenario 7 as an example. Modify `scen_name` and `scen_path` for your scenario.
scen_name = 'rs7_capacity'
scen_path = r'C:\FTOT\scenarios\reference_scenarios\rs7_capacity'

# TODO: use GDB instead for the road network layer
shp_path = os.path.join(scen_path, 'temp_networkx_shp_files')

picklename = os.path.join(scen_path, 'BetweenessG.pickle')

# TODO: Set to true if base scenario has background flows set to true in the scenario XML
BACKGROUND_FLOWS = False

# TODO: may not require this anymore though another code change is decreasing buffer from 100 to 50 miles
if not os.path.exists(shp_path):
    print('Please modify the FTOT code using the `ftot_networkx.py` and `ftot_routing.py` scripts in this repository and run the scenario again.')

In [2]:
# Read in prepared betweeness centrality and road network graph data
# If these don't exist, the following steps will create them
if os.path.exists(picklename):
    file = open(picklename, 'rb')
    betweenness_dict_road = pickle.load(file)
    G_road = pickle.load(file)

In [3]:
# Start by using betweenness centrality calculation using networkX
if not os.path.exists(picklename):
    # TODO: replace use of geopandas with ogr from osgeo which is already in this environment (see read_gdb method in FTOT as an example)
    road = gpd.read_file(os.path.join(shp_path, 'road.shp'))
    
    # convert from geodataframe to Graph for networkX
    G_road = momepy.gdf_to_nx(road, approach='primal')
    
    # Process the networkX graph
    G_road = nx.convert_node_labels_to_integers(G_road, first_label=0, ordering='default', label_attribute="xy_coord_label")

In [4]:
# Run betweenness centrality on the NetworkX graph
# Note: This step might take several minutes to a few hours
# Run if pickle not available
# TODO: can likely skip this step if running 'V' not 'BC' but then do not have betweenness_dict_road variable
if not os.path.exists(picklename):
    print('Running Betweenness Centrality calculations. This might take more than 20 minutes.')
    betweenness_dict_road = nx.betweenness_centrality(G_road, normalized=False, weight='Length')
    print('Completed Betweenness Centrality calculations.')

In [5]:
# Save with pickle
# On load, need to know that there are two objects in this pickle, the betweenness centrality dict and the network G
if not os.path.exists(picklename):
    with open(picklename, 'wb') as handle:
        pickle.dump(betweenness_dict_road, handle)
        pickle.dump(G_road, handle)

## Join Betweenness Centrality calculations to edges

- Sum BC for each node of a link
- Create data frame for repeated link removal

In [6]:
# Read in FTOT data
print(scen_path)
db_name = 'main.db'

db_path = 'sqlite:///' + os.path.join(scen_path, db_name)

engine = sqlalchemy.create_engine(db_path)

table_name = 'networkx_edges'
nx_edges = pd.read_sql_table(table_name, engine)

# TODO: bring in additional metrics (route_cost, transport_cost, route_cost_transport, co2_cost) from networkx_edge_costs table

table_name = 'networkx_nodes'
nx_nodes = pd.read_sql_table(table_name, engine)

table_name = 'optimal_variables'
optimal_vars = pd.read_sql_table(table_name, engine)

C:\FTOT\scenarios\reference_scenarios\rs7_capacity


In [7]:
optimal_vars

variable_type  var_id  variable_value  converted_capacity  \
0            Edge       1     181436.9500                 NaN   
1            Edge       2       9071.8474                 NaN   
2            Edge       3      22679.6180                 NaN   
3            Edge       4      68038.8560                 NaN   
4            Edge       5      22679.6180                 NaN   
..            ...     ...             ...                 ...   
492          Edge   34075      58967.0080                 NaN   
493          Edge   34081      58967.0080       704385.369727   
494          Edge   34083      81646.6270       728794.816589   
495          Edge   34084     181436.9500                 NaN   
496          Edge   34086      99790.3210       728794.816589   

     converted_volume  converted_capac_minus_volume  edge_type  \
0                 NaN                           NaN  connector   
1                 NaN                           NaN  connector   
2                 NaN                           NaN  connector   
3                 NaN                           NaN  connector   
4                 NaN                           NaN  connector   
..                ...                           ...        ...   
492               0.0                           NaN  transport   
493          312744.0                 391641.369727  transport   
494          206676.0                 522118.816589  transport   
495               0.0                           NaN  transport   
496          206676.0                 522118.816589  transport   

     commodity_name  o_facility  d_facility  ...       units  variable_name  \
0      freight_bulk   rmp_25017   rmp_25017  ...  metric_ton         Edge_1   
1    freight_parcel  dest_25005  dest_25005  ...  metric_ton         Edge_2   
2    freight_parcel  dest_25009  dest_25009  ...  metric_ton         Edge_3   
3    freight_parcel  dest_25021  dest_25021  ...  metric_ton         Edge_4   
4    freight_parcel  dest_25023  dest_25023  ...  metric_ton         Edge_5   
..              ...         ...         ...  ...         ...            ...   
492  freight_parcel        None  dest_25027  ...  metric_ton     Edge_34075   
493  freight_parcel        None        None  ...  metric_ton     Edge_34081   
494  freight_parcel        None        None  ...  metric_ton     Edge_34083   
495    freight_bulk        None  proc_25025  ...  metric_ton     Edge_34084   
496  freight_parcel        None        None  ...  metric_ton     Edge_34086   

     nx_edge_id  mode  mode_oid    length  original_facility final_facility  \
0           NaN  None       NaN       NaN               None           None   
1           NaN  None       NaN       NaN               None           None   
2           NaN  None       NaN       NaN               None           None   
3           NaN  None       NaN       NaN               None           None   
4           NaN  None       NaN       NaN               None           None   
..          ...   ...       ...       ...                ...            ...   
492     17043.0  road  311232.0  0.036034               None           None   
493     17046.0  road  311217.0  0.000161               None           None   
494     17047.0  road  311220.0  0.075613               None           None   
495     17048.0  road  311234.0  0.239426               None           None   
496     17049.0  road  311219.0  0.217271               None           None   

    prior_edge distance_travelled  
0         None               None  
1         None               None  
2         None               None  
3         None               None  
4         None               None  
..         ...                ...  
492       None               None  
493       None               None  
494       None               None  
495       None               None  
496       None               None  

[497 rows x 28 columns]

In [8]:
# TODO: replace use of geopandas with ogr from osgeo which is already in this environment (see read_gdb method in FTOT as an example)
road_orig_label = gpd.read_file(os.path.join(shp_path, 'road.shp'))
# convert from geodataframe to Graph for networkX
G_road_orig_label = momepy.gdf_to_nx(road_orig_label, approach='primal')

In [9]:
road_orig_label_nodes = list(G_road_orig_label.nodes) # these values are the shape_x and shape_y values in `networkx_nodes` 
# Use that to get node_id from networkx_edges in the database
# Then use those id values to get edges info
# Then line up the new integer labels with this list of ids to get betweenness centrality for each node

In [10]:
# Make the betweenness_centrality values as the framework to join in shape_x, shape_y, and node_id
# TODO: can likely skip this step if using 'V' not 'BC'
bc_df_road = pd.DataFrame.from_dict(betweenness_dict_road, orient = 'index')
bc_df_road = bc_df_road.rename(columns = {0: 'BC'})

In [11]:
node_shape_df_road = pd.DataFrame(road_orig_label_nodes)

# TODO: can likely skip this step if using 'V' not 'BC' but may complicate things if missing shape_x and shape_y
bc_shape_df_road = pd.concat([bc_df_road, node_shape_df_road], axis = 1)
bc_shape_df_road = bc_shape_df_road.rename(columns = {0: 'shape_x', 1: 'shape_y'})

# Now add node_id from networkx_nodes, using pandas merge with left join
# Use both shape_x and shape_y to identify the nodes correctly

bc_node_df = pd.merge(bc_shape_df_road, nx_nodes, on = ['shape_x', 'shape_y'], how = 'left')

In [12]:
# Now use this dataframe to populate a dataframe of edges
# We will want the following from networkx_edges:
# edge_id, from_node_id, to_node_id, mode_source, miles, mode_source_oid
# Then using the node_id column in the new bc_node_df, add these:
# from_node_BC, to_node_BC
# and sum those for sum_node_BC
# TODO: can likely skip this step if using 'V' not 'BC'
merge_from = pd.merge(nx_edges, bc_node_df[['BC', 'node_id']],
                      left_on = 'from_node_id',
                      right_on = 'node_id',
                      how = 'left')
merge_from = merge_from.rename(columns = {'BC': 'from_node_BC'})

merge_to = pd.merge(merge_from, bc_node_df[['BC', 'node_id']],
                    left_on = 'to_node_id',
                    right_on = 'node_id',
                    how = 'left')
merge_to = merge_to.rename(columns = {'BC': 'to_node_BC'})

# Sum the BC values

merge_to['sum_BC'] = merge_to.filter(like = "node_BC").sum(axis = 1)

# Then from optimal_variables, get variable_name, nc_edge_id, mode, mode_oid, miles,
# variable_value, converted_capacity, and converted_volume

use_opt_vars = ['variable_type',
               'var_id',
               'variable_value',
                'variable_name',
                'nx_edge_id',
                'mode_oid',
                'converted_capacity',
                'converted_volume'
               ]

merge_opt = pd.merge(merge_to, optimal_vars[use_opt_vars],
                     left_on = 'edge_id',
                     right_on = 'nx_edge_id',
                     how = 'left')

In [ ]:
merge_to

In [ ]:
optimal_vars[use_opt_vars]

In [ ]:
merge_opt.head(3)

In [ ]:
# Create ranked lists of edges to remove
# First, keep only edges in the optimal solution
# Then rank by sum_BC
# Then just keep the columns we need, and reset the index
# Note: in resiliency_disruptions.disrupt_network, the edges_remove DataFrame is sorted again by 'V' or 'BC'
# TODO: update use_cols as needed for additional metrics
use_cols = ['edge_id', 'from_node_id', 'to_node_id', 'length', 'capacity', 'volume', 'sum_BC',
            'variable_type', 'variable_value', 'variable_name', 'nx_edge_id', 'mode_oid', 'converted_capacity',
            'converted_volume']

# TODO: if 'V', we may not have sum_BC column at this point
# TODO: note that sort_values (both here and in resiliency_disruptions.disrupt_network) does not break ties (significant for 'V')
edges_remove = merge_opt[merge_opt['variable_value'] > 0].sort_values(by = 'sum_BC', ascending = False).filter(items = use_cols).reset_index()

edges_remove.to_csv(os.path.join(scen_path, 'Edges_to_Remove.csv'),
                    index = False)

edges_remove.head(3)

## Create Scenarios, Disrupt, Run FTOT

Create disrupted network by copying everything in `scen_path` to a new directory.

Then overwrite the `networkx_edges` tables in that main.db with the disrupted versions.

##### Assumptions:

  1. ArcGIS with 64-bit geoprocessing is installed.
  2. The FTOT version being used has been modified according to the `README` in this directory.


In [ ]:
disrupt_type = 'BC' # Can disrupt based on betweenness centrality, 'BC', or volume, 'V'
disrupt_steps = 12  # This is the number of steps to use. Recommend at least 25.

resiliency_disruptions.make_disruption_scenarios(disrupt_type, disrupt_steps, scen_path)

In [ ]:
resiliency_disruptions.disrupt_network(disrupt_type, disrupt_steps, scen_path, edges_remove)

In [ ]:
# TODO: move these higher up in the notebook
PYTHON = r"C:\FTOT\python3_env\python.exe"
repo_location = %pwd
repo_location = os.path.split(repo_location)[0]
FTOT = r"C:\FTOT\program\ftot.py"  # Optionally: os.path.join(repo_location, 'program', 'ftot.py')
print(FTOT)

In [ ]:
# Begin running O through M steps of FTOT on the disupted scenarios
# This may take several hours, depending on size of the network and number of steps

results = resiliency_disruptions.run_o_steps(disrupt_type, disrupt_steps, scen_path, PYTHON, FTOT)

#### Optional: Repeat with volume-based disruptions

Creates a separate directory tree for the volume-based disruptions and carries out the disruption steps on that set.

Set the variable `DO_VOLUME` to `True` to run the following steps.

In [ ]:
DO_VOLUME = True

if DO_VOLUME:

    disrupt_type = 'V'
    disrupt_steps = 5

    resiliency_disruptions.make_disruption_scenarios(disrupt_type, disrupt_steps, scen_path)
    resiliency_disruptions.disrupt_network(disrupt_type, disrupt_steps, scen_path, edges_remove)
    results = resiliency_disruptions.run_o_steps(disrupt_type, disrupt_steps, scen_path, PYTHON, FTOT)
    results

## Generate disruption result report

In [ ]:
# TODO: bug in the handoff here, but not reported out to the user... looks like it succeeded but it did not
# TODO: hand in the disrupt_type parameter as well (right now, hard-coded in RMarkdown as BC)
R_process = subprocess.Popen(['Rscript.exe', 'compile_report.R', scen_path, BACKGROUND_FLOWS, DO_VOLUME,],
                             stdout = subprocess.PIPE, stderr = subprocess.PIPE)

here = os.getcwd()
webbrowser.open('file://' + os.path.realpath(os.path.join(here, 'Disruption_Results.html')))

# TODO: move html file to scen_path

In [ ]:
os.path.realpath(os.path.join(here, 'Disruption_Results.html'))